In [26]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]= "3"

import pickle as pkl

import torch
# device = "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [27]:
# Unpickle the pre-processed training data
train_data_file = f"../../data/processed_data/train_df.pkl"
with open(train_data_file, 'rb') as f:
    train_df = pkl.load(f)

# Unpickle the pre-processed validation data
val_data_file = f"../../data/processed_data/val_df.pkl"
with open(val_data_file, 'rb') as f:
    val_df = pkl.load(f)

In [28]:
label_number_map = {
    "000":"0",
    "100":"1",
    "101":"2",
    "110":"3",
    "111":"4",
}
def new_label(hs, ag, tr):
    if hs == 0:
        return label_number_map.get("000")
    elif ag == 0 and tr == 0:
        return label_number_map.get("100")
    elif ag == 0 and tr == 1:
        return label_number_map.get("101")
    elif ag == 1 and tr == 0:
        return label_number_map.get("110")
    elif ag == 1 and tr == 1:
        return label_number_map.get("111")
    else:
        raise Exception

In [29]:
train_df['class_label'] = train_df.apply(lambda x: new_label(x.HS, x.AG, x.TR), axis=1)
train_df = train_df[["raw_text", "class_label"]].rename(columns={"raw_text": "body", "class_label": "target"})
train_df
# train_df = train_df[:200]

,body,target
id,,
201,"Hurray, saving us $$$ in so many ways @potus @...",1
202,Why would young fighting age men be the vast m...,1
203,@KamalaHarris Illegals Dump their Kids at the ...,1
204,NY Times: 'Nearly All White' States Pose 'an A...,0
205,Orban in Brussels: European leaders are ignori...,0
...,...,...
9196,@mmdwriter @JRubinBlogger @BenSasse I am proud...,0
9197,@CheriJacobus Hollywood is complicit in the ra...,0
9198,@amaziah_filani What a fucking cunt I hate see...,4


In [30]:
val_df['class_label'] = val_df.apply(lambda x: new_label(x.HS, x.AG, x.TR), axis=1)
val_df = val_df[["raw_text", "class_label"]].rename(columns={"raw_text": "body", "class_label": "target"})
val_df
# val_df = val_df[:20]


,body,target
id,,
18201,I swear I’m getting to places just in the nick...,0
18202,I’m an immigrant — and Trump is right on immig...,0
18203,#IllegalImmigrants #IllegalAliens #ElectoralSy...,3
18204,@DRUDGE_REPORT We have our own invasion issues...,3
18205,Worker Charged With Sexually Molesting Eight C...,0
...,...,...
19196,@SamEnvers you unfollowed me? Fuck you pussy,0
19197,@DanReynolds STFU BITCH! AND YOU GO MAKE SOME ...,4
19198,"@2beornotbeing Honey, as a fellow white chick,...",0


In [31]:
df_train, df_val = train_df, val_df

In [32]:
from transformers import AutoTokenizer
PRE_TRAINED_MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


In [33]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

bert_model

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [34]:

token_lens = []
for txt in df_train.body:
  tokens = tokenizer.encode(txt, max_length=128)
  token_lens.append(len(tokens))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [35]:
import numpy as np
MAX_LEN = np.max(token_lens)
BATCH_SIZE = 32

In [36]:
from torch.utils.data import Dataset, DataLoader

class SATweetDataset(Dataset):

  def __init__(self, tweets, targets, tokenizer, max_len):
    self.tweets = tweets
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.tweets)
  
  def __getitem__(self, item):
    tweet = str(self.tweets[item])
    target = int(self.targets[item])

    encoding = self.tokenizer.encode_plus(
      tweet,
      add_special_tokens=True,
      max_length=self.max_len,
      truncation= True,
      return_token_type_ids=False,
      padding = 'max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'tweet_text': tweet,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
     

In [37]:

def create_data_loader(df, tokenizer, max_len, batch_size):

    ds = SATweetDataset(
        tweets=df.body.to_numpy(),
        targets=df.target.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=4
    )

In [38]:

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)

In [39]:
train_data_loader

In [40]:

data = next(iter(train_data_loader))
data.keys()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

dict_keys(['tweet_text', 'input_ids', 'attention_mask', 'targets'])

In [41]:

n_classes = 5
class_names = ['1', '2', '3', '4', '5']
PRE_TRAINED_MODEL_NAME = "roberta-base"
WEIGHTS_PATH = "roberta_weights.bin"
n_classes = 5

In [42]:

input_ids = data['input_ids']
attention_mask = data['attention_mask']

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([32, 128])
torch.Size([32, 128])


In [43]:
from torch import nn, optim

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    #importing the pre_trained model from transformers library
    self.bert = bert_model.from_pretrained(PRE_TRAINED_MODEL_NAME)
    #Adding a linear layer on the top of the bert model
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  #The forward step function
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False
    )
    output = pooled_output
    return self.out(output)

In [44]:
model = SentimentClassifier(n_classes)
model.cpu()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SentimentClassifier(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [45]:
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location="cpu"))

<All keys matched successfully>

In [46]:
def eval_model(model, data_loader, loss_fn, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0
  all_preds, all_targets = [], []

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"]
      attention_mask = d["attention_mask"]
      targets = d["targets"]

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())
      all_preds.append(preds)
      all_targets.append(targets)

  return correct_predictions.double() / n_examples, np.mean(losses), all_preds, all_targets

In [47]:
loss_fn = nn.CrossEntropyLoss()
val_acc, val_loss, preds, targets = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    len(df_val)
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [48]:
val_acc

tensor(0.6650, dtype=torch.float64)

In [49]:
len(preds), len(preds[0])

(32, 32)

In [50]:
len(targets)

32

In [51]:
len(val_df)

1000

In [52]:
prediction_vals, target_vals = [], []
for tensor in preds:
    for val in tensor:
        prediction_vals.append(val.item())
for tensor in targets:
    for val in tensor:
        target_vals.append(val.item())

prediction_list, target_list = prediction_vals[:len(val_df)], target_vals[:len(val_df)]
        

In [53]:
match, miss = 0,0
for prediction, target in zip(prediction_list, target_list):
    if prediction == target:
        match+=1
    else:
        miss+=1

In [54]:
match, miss

(665, 335)

In [55]:
torch.cuda_is_available()

AttributeError: module 'torch' has no attribute 'cuda_is_available'

In [ ]:
os.environ